In [ ]:
import os
from os.path import join
import numpy as np
import argparse
from timeit import default_timer as timer

import torch
from nuscenes.nuscenes import NuScenes

import glob

from nuscenes.utils.data_classes import RadarPointCloud
from nuscenes.utils.geometry_utils import view_points, transform_matrix

from pyquaternion import Quaternion


In [ ]:
DIR_DATA = "/ssd/Datasets_and_code/nuscenes_depth_estimation/dataset/dataset_radar_cam"
DIR_NUSCENES = "/ssd/Datasets_and_code/nuscenes_depth_estimation/dataset/nuscenes_mini"
VERSION = "v1.0-mini"

In [ ]:
nusc = NuScenes(VERSION, dataroot = DIR_NUSCENES, verbose=False)

dir_data_out = join(DIR_DATA, 'radar')
if not os.path.exists(dir_data_out):
    os.makedirs(dir_data_out)

# Remove all files in the output folder
f_list=glob.glob(join(dir_data_out,'*'))
for f in f_list:
    os.remove(f)
print('removed %d old files in output folder' % len(f_list))

In [ ]:
sample_indices = torch.load(join(DIR_DATA,'data_split.tar'))['all_indices']
       
N_total = len(sample_indices)
print('Total sample number:', N_total)

In [ ]:
for sample_idx in sample_indices:
    
    sample_rec = nusc.sample[sample_idx]
    radar_token = sample_rec['data']['RADAR_FRONT']        
    radar_sample = nusc.get('sample_data', radar_token)
    
    pcl_path = os.path.join(nusc.dataroot, radar_sample['filename'])
    RadarPointCloud.disable_filters()
    pc = RadarPointCloud.from_file(pcl_path)
    
    cam_token = nusc.sample[sample_idx]['data']['CAM_FRONT']
    cam = nusc.get('sample_data', cam_token)
    
    # Points live in the point sensor frame. So they need to be transformed via global to the image plane.
    # First step: transform the pointcloud to the ego vehicle frame for the timestamp of the sweep.
    cs_record = nusc.get('calibrated_sensor', radar_sample['calibrated_sensor_token'])
    pc.rotate(Quaternion(cs_record['rotation']).rotation_matrix)
    pc.translate(np.array(cs_record['translation']))

    # Second step: transform from ego to the global frame.
    poserecord = nusc.get('ego_pose', radar_sample['ego_pose_token'])
    pc.rotate(Quaternion(poserecord['rotation']).rotation_matrix)
    pc.translate(np.array(poserecord['translation']))

    # Third step: transform from global into the ego vehicle frame for the timestamp of the image.
    poserecord = nusc.get('ego_pose', cam['ego_pose_token'])
    pc.translate(-np.array(poserecord['translation']))
    pc.rotate(Quaternion(poserecord['rotation']).rotation_matrix.T)

    # Fourth step: transform from ego into the camera.
    cs_record = nusc.get('calibrated_sensor', cam['calibrated_sensor_token'])
    pc.translate(-np.array(cs_record['translation']))
    pc.rotate(Quaternion(cs_record['rotation']).rotation_matrix.T)
    
    np.save(join(dir_data_out, '%05d_radar_pc' % sample_idx), pc.points[:3,:])